# WasteNet

## Setup

In [ ]:
!rm -r smart-cities-drl
!git clone https://github.com/eescriba/smart-cities-drl
!cd smart-cities-drl/ && pip install -e .
!pip install ray[rllib]

In [ ]:
import json
import shutil

import gym
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune import run, sample_from
from ray.tune.schedulers import PopulationBasedTraining

import sys
sys.path.insert(0,'./smart-cities-drl/src/')
from wastenet.env import WasteNetEnv

## Proximal Policy Optimization (PPO)

In [ ]:
ray.shutdown()
ray.init(ignore_reinit_error=True)

CHECKPOINT_ROOT = "tmp/ppo/wastenet"
shutil.rmtree(CHECKPOINT_ROOT, ignore_errors=True, onerror=None)

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 1

agent = ppo.PPOTrainer(config, env=WasteNetEnv)

N_ITER = 1000
results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(CHECKPOINT_ROOT)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

In [ ]:
policy = agent.get_policy()
model = policy.model
print(model.base_model.summary())

In [ ]:
obs = env.reset()
done = False
episode_reward = 0
sum_reward = 0
n_step = 1000
for step in range(n_step):
    action = agent.compute_action(obs)
    print(action)
    state, reward, done, info = env.step(action)
    print(state, reward, done, info)
    sum_reward += reward
    if done:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

## Tuned PPO

### Population Based Training (PBT)

In [ ]:
# Postprocess the perturbed config to ensure it's still valid used if PBT.
def explore(config):
    # ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["sgd_minibatch_size"] * 2:
        config["train_batch_size"] = config["sgd_minibatch_size"] * 2
    # ensure we run at least one sgd iter
    if config["num_sgd_iter"] < 1:
        config["num_sgd_iter"] = 1
    return config

pbt = PopulationBasedTraining(
    time_attr="time_total_s",
    perturbation_interval=120,
    resample_probability=0.25,
    metric="episode_reward_mean",
    mode="max",
    # Specifies the mutations of these hyperparams
    hyperparam_mutations={
        "lambda": lambda: random.uniform(0.9, 1.0),
        "clip_param": lambda: random.uniform(0.01, 0.5),
        "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
        "num_sgd_iter": lambda: random.randint(1, 30),
        "sgd_minibatch_size": lambda: random.randint(128, 16384),
        "train_batch_size": lambda: random.randint(2000, 160000),
    },
    custom_explore_fn=explore)

In [ ]:
analysis = run(
        'PPO',
        name="prueba_ppo",
        scheduler=pbt,
        num_samples=8,
        config={
            "env": WasteNetEnv,
            "seed": 123,
            "kl_coeff": 1.0,
            "num_gpus": 1,
            "num_workers": 1,
            "observation_filter": "MeanStdFilter",
            "model": {
                # "fcnet_hiddens": [
                #     32,
                #     32
                # ],
                "free_log_std": True
            },
            "num_sgd_iter": 10,
            "sgd_minibatch_size": 128,
            "lambda": sample_from(lambda spec: random.uniform(0.9, 1.0)),
            "clip_param": sample_from(lambda spec: random.uniform(0.1, 0.5)),
            "lr": sample_from(lambda spec: random.uniform(1e-3, 1e-5)),
            "train_batch_size": sample_from(
                lambda spec: random.randint(1000, 60000))
        })
print("best hyperparameters: ", analysis.best_config)

## Evaluation

In [ ]:
!rllib rollout ...

In [ ]:
!tensorboard --logdir=$HOME/ray_results/